## Objetivo y alcance




Para esta materia el objetivo es poder hacer un primer acercamiento a un proceso de aprendizaje supervisado.

En este laboratorio no se espera que se encuentre el mejor modelo con sus mejores parámetros, sino que se logre la buena práctica de realizar los pasos necesarios en un proceso de aprendizaje supervisado, desde el entendimiento de la pregunta de investigación hasta la preparación de datos, desde la división del dataset hasta la evaluación del modelo. Para realizar el práctico vamos a utilizar el datasets que hayan generado en el práctico anterior.


## Introducción
En un mismo día une operador humano de una clínica responde cientos de consultas sobre problemáticas increiblemente sensibles como lo es la salud de una persona y las preocupaciones relacionadas a eso. Además la demanda de trabajo de un operador humano es multivariada y dinámica: algunos días hay un flujo mayor de pacientes usando los sistemas hospitalarios, y hay momentos del mes y del día en donde prácticamente no hay ninguna consulta a atender. 

El dato de **tiempo de respuesta de HH** tiene muchas implicaciones a nivel producto. Tener ese dato nos permite:
- informarle a la persona usuaria el tiempo promedio de respuesta antes de pasarla a hablar con une operador, para que adecúe sus expectativas a la realidad. 
- informarle detalladamente este dato a los hospitales como reporte de uso al final del mes, para que organice al grupo de trabajo eficientemente y con información veraz
- permitir configurar lógicas automáticas de repartición de tareas para les operadores, para facilitarles su trabajo

Por ello lo que se pretende en este práctico es poder armar un modelo que nos ayude a predecir el tiempo de respuesta de HH. Lo que nos da este dato es la presencia de la feature `Got_HH`, que cuando vale 1 nos marca en el dataset que el bot realizó la transferencia. El siguiente mensaje que tenga `Direction == outbound` sería el primer mensaje de un operador humano. La diferencia de horas entre estos dos mensajes nos da el tiempo de respuesta de un operador humano.

Predecir esta variable teniendo en cuenta las distintas features del dataset será una tarea de regresión. Entrenar un modelo por hospital dará una mejor predicción. Por conocimiento del dataset sabemos que entrenar por porción del día (mañana/tarde) también dará mejores resultados, pero esto es opcional.

## Tareas
En orden,

Deberán armar un dataset más pequeño que el actual, **sacando**:
- las filas de los hospitales sin HH (Demo, Desarrollo, Clínica Carrá, Default) y las conversaciones sin HH (que agrupadas por conv_id no tengan un Got_HH == 1)
- las conversaciones que nunca hayan pasado a HH (es decir Got_HH es 0 en toda la agrupación por conv_id) (guiarse con https://stackoverflow.com/questions/52393659/pandas-dataframe-check-if-column-value-exists-in-a-group-of-columns )
- Para simplificar, las conversaciones que hayan pasado a HH pero el operador nunca respondió. Es decir Got_HH es 1 en alguna fila, pero no hay una fila siguiente con direction igual a outbound.

**Generar** las features:

- `tiempo_de_respuesta_HH` (agrupando por conv_id, calcular el valor absoluto de la diferencia entre el SentDate del mensaje con Got_HH igual a 1 y el del siguiente mensaje con Direction igual a outbound https://stackoverflow.com/questions/34023918/make-new-column-in-panda-dataframe-by-adding-values-from-other-columns )
- cant de mensajes de la conversación (agrupación por conv_id+dia y luego sum) (https://stackoverflow.com/questions/39922986/how-do-i-pandas-group-by-to-get-sum)
- momento del dia (day_moment fue generada por el grupo 2 y puede reutilizarse el código que está aquí https://colab.research.google.com/drive/1KHgwiRqFLlavfhnCNdVvqG-xoPc0qMsM?usp=sharing#scrollTo=P04lwj2bSrFM )

**manteniendo** las features:

> - dia de la semana
> - fecha
> - presencia de errores en la conversación
> - hospital
> - cancelled
> - consulted
> - no correlation
> - y en general, todas las features que no sean body ni direction.
> - Debiendo quedar una fila por conversación.

* Cargar los datos, separando del dataset la etiqueta a predecir.
* Dividir el dataset en el conjunto de entrenamiento y conjunto de test
* Elegir y fundamentar si usar regresión lineal o polinomial(https://towardsdatascience.com/introduction-to-linear-regression-and-polynomial-regression-f8adc96f31cb, https://data36.com/polynomial-regression-python-scikit-learn/ ) . Tomarse un tiempito en hacer esta decisión les será muy útil ya que es una decisión interesante para comunicar en el video que tengan que hacer a partir de este práctico.
* Entrenar y evaluar un modelo de regresión lineal y uno polinomial, fijando la semilla aleatoria para hacer repetible el experimento.
* En cuanto a los hiper-parámetros:

        1.   Probar primero con los default y elegir alguna/s métrica/s para reportar los resultados. 
        2.   Luego usar grid-search (https://machinelearningmastery.com/hyperparameter-optimization-with-random-search-and-grid-search/) para explorar muchas combinaciones posibles de valores (Ejemplo de uso de grid search para encontrar el grado de polynomial regression https://stackoverflow.com/a/62460485/13597482 ). Reportar MAE y RMSE para las variaciones que busquen.

*   Para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando MAE y RMSE.

### Se evaluarán los siguientes aspectos:
  ***1-*** Que se apliquen los conceptos vistos con los profes en el teórico y en el práctico.

  ***2-*** Capacidad de análisis. Enfocarse en esto les permitirá completar el video que deben preparar a continuación.

  ***3-*** Criterio para elegir que solución aplicar en cada caso y con qué método implementarla.


  
## Deadline pautado para la entrega: Miércoles 21/09/2022

Es importante que vayan anotando **conclusiones y observaciones en lenguaje natural** (no muy extensas) a medida que vayan realizando las tareas para que les sirva como **punto de partida para el video** de 10 min que se realice a partir de este práctico. El video que sigue a este práctico serviría como informe de resultados y observaciones de este práctico, resaltando aprendizajes y problemáticas con las que se hayan encontrado.

Lo importante de este práctico **no es encontrar el mejor modelo posible** ni mucho menos, sino poder ejercitar todo el proceso, que implica tanto **preparar los datos hasta correr los experimentos**, y finalmente **analizar y explicar los resultados encontrados**. En mi experiencia, la primera reacción ante saber que hay que comunicar resultados es querer comunicar éxitos, pero los resultados exitosos son -la mayoría de las veces- poco informativos. En el proceso de realización del práctico se encontrarán con desafíos de las tareas propuestas y con problemas específicos de este tipo de datasets conversacionales. Esa información es la valiosa a comunicar ya que es una experiencia que ningún otro grupo pudo tener, y le puede servir a otro grupo en un futuro que se relacione con datos que tengan características parecidas.

# Resolución

In [88]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib
import matplotlib.pyplot as plt



In [89]:
file_key_marzo = 'marzo_anon_test.csv'
file_key_febrero = 'febrero_anon_test.csv'

file_key_procesado = 'dataset_post_entrega_2.csv'

df_marzo = pd.read_csv(file_key_marzo)
df_febrero = pd.read_csv(file_key_febrero)

In [90]:
df = pd.concat([df_febrero, df_marzo]).drop(['Unnamed: 0'], axis=1)
df['SentDate'] = pd.to_datetime(df.SentDate)

In [91]:
df.sample(2)

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id
34378,Carra,whatsapp:+5493515266217,Ya te encontrás registrado.,read,2022-02-04 08:27:00+00:00,2022-02-04,4,2,8,1,...,0,[],0,[],0,[],0,[],0,1184
51125,Carra,whatsapp:+5493515266217,8075.0,received,2022-03-07 18:26:54+00:00,2022-03-07,7,3,18,1,...,0,[],0,[],0,[],0,[],0,1629


In [92]:
df.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id'],
      dtype='object')

In [93]:
df.Dia.value_counts().reset_index().sort_values(by='index')

,index,Dia
22,1,4444
3,2,26475
6,3,23762
15,4,17148
23,5,3861
27,6,2598
2,7,29068
4,8,24688
13,9,20032
12,10,20222


### ¿Cómo se ven los mensajes de _Human Handoff_?
Buscamos una muestra de los mensajes que nos interesan, solo para ver que forma tienen

In [94]:
df_handoffed = df[df['Got_HH'] == 1]
df_handoffed[['conv_id', 'Body', 'Hospital', 'Direction']]

,conv_id,Body,Hospital,Direction
164,10,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
408,26,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
518,32,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
609,34,"Fuiste transferido a un humano, por favor escr...",Salud Consultorios,outbound-api
666,25,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
...,...,...,...,...
290219,4596,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290339,7341,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290660,7056,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api
290921,7325,"Fuiste transferido a un humano, por favor escr...",CityBell,outbound-api


In [95]:
df_handoffed.Direction.value_counts()

outbound-api    2756
inbound            1
Name: Direction, dtype: int64

In [96]:
# Esto es un error ? 
df_handoffed[df_handoffed.Direction == 'inbound']

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,Falla_Api_Externo,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id
197335,CityBell,whatsapp:+5492215137770,"Fuiste transferido a un humano, por favor escr...",received,2022-03-22 12:07:49+00:00,2022-03-22,22,3,12,1,...,0,[],0,[],0,[],0,[],0,4238


## ¿Cómo se vé una conversación que finaliza en _Human Handoff_?

Seleccionamos un valor de `conv_id` y `Hospital` como muestra para poder calcular el valor

In [97]:
# Nota:
# El valor de conv_id no es único.
df[df['conv_id'] == 32].Hospital.value_counts()

CityBell    56
Carra       22
Name: Hospital, dtype: int64

### Boceto: Calcular tiempo de respuesta para una única conversación

In [98]:
# Elegimos una conversación cualquiera, vemos que intercambio de mensajes hay

def get_sample(df, **kwargs):
    return df[(df['conv_id'] == kwargs['conv_id']) & (df['Hospital'] == kwargs['hospital'])][['Body','Direction','SentDate','Got_HH', 'Hospital']].sort_values(by='SentDate')


sample_conv_city_bell = get_sample(df, hospital = 'CityBell', conv_id = 32)
sample_conv_city_bell


,Body,Direction,SentDate,Got_HH,Hospital
6442,XXX,inbound,2022-02-16 11:45:35+00:00,0,CityBell
6437,XXXXXXXXXXXXX,inbound,2022-02-16 11:45:44+00:00,0,CityBell
6436,"Hola, soy el asistente virtual de Centro Medic...",outbound-api,2022-02-16 11:45:45+00:00,0,CityBell
6422,Sacar un turno,inbound,2022-02-16 11:46:04+00:00,0,CityBell
6421,"Necesito tu DNI, por favor.",outbound-api,2022-02-16 11:46:04+00:00,0,CityBell
6411,XXXXXXXX,inbound,2022-02-16 11:46:14+00:00,0,CityBell
6409,No se encuentra registrado ese DNI en el siste...,outbound-api,2022-02-16 11:46:16+00:00,0,CityBell
6408,¿Te puedo ayudar con algo más?,outbound-api,2022-02-16 11:46:16+00:00,0,CityBell
6395,Quiero hablar con un operador,inbound,2022-02-16 11:46:48+00:00,0,CityBell
6394,Has seleccionado una práctica que requiere ate...,outbound-api,2022-02-16 11:46:48+00:00,0,CityBell


In [99]:
# 7        Salud Consultorios
sample_conv_salud_c = get_sample(df, hospital = 'Salud Consultorios', conv_id = 7)
sample_conv_salud_c

,Body,Direction,SentDate,Got_HH,Hospital
173,XXXX,inbound,2022-03-01 19:33:58+00:00,0,Salud Consultorios
171,"Hola, soy el asistente virtual de Salud Consul...",outbound-api,2022-03-01 19:33:59+00:00,0,Salud Consultorios
165,Sacar turno,inbound,2022-03-01 19:34:20+00:00,0,Salud Consultorios
163,"Necesito tu DNI, por favor.",outbound-api,2022-03-01 19:34:21+00:00,0,Salud Consultorios
159,XXXXXXXXX,inbound,2022-03-01 19:34:28+00:00,0,Salud Consultorios
...,...,...,...,...,...
146584,No se pudo enviar el mensaje,outbound-api,2022-03-16 14:26:36+00:00,0,Salud Consultorios
146591,"Necesito hacer estos análisis, me podrían pasa...",inbound,2022-03-16 14:27:12+00:00,0,Salud Consultorios
146534,201.0,inbound,2022-03-16 14:30:12+00:00,0,Salud Consultorios
146234,Buenas tardes! para laboratorio debe comunicar...,outbound-api,2022-03-16 14:47:47+00:00,0,Salud Consultorios


In [100]:
def SentDate_delta(event, posteriores):
    """
    retorna el 
    """
    to = posteriores[(posteriores.Direction == 'outbound-api') & (posteriores.Got_HH == 0)]
    if not len(to):
        return None
    
    return to.head(1).SentDate - event.SentDate

# hh_machinery[T](data: DataFrame, handler: (t, []t): T): T 
def hh_machinery(data, handler = SentDate_delta):
    """
    hh_machinery[T](data: DataFrame, handler: (t, []t): T): T 
    data es un conjunto de mensajes pertenecientes a una misma conversación (para un mismo día y hospital)
    handler es una funcion que puede recibir un mensaje de human handoff y los posteriores a ese
    Retorna lo que devuelva handler: 
    
    Pre: data tiene al menos un mensaje con Got_HH en 1, los mensajes estan ordenados por SentDate
    """
    conv = data.reset_index()

    # Pre:conv tiene al menos un mensaje con Got_HH en 1
    handoff = conv[conv.Got_HH == 1]
    idx = handoff.index[0] 
    
    # Si es el último mensaje, no podemos obtener el tiempo buscado
    if idx == len(handoff):
        return None
        
    return handler(conv.iloc[idx], conv[idx:])

tiempo_de_respuesta_HH = lambda data: hh_machinery(data)

In [101]:
t2hh = tiempo_de_respuesta_HH(sample_conv_city_bell)
print (type(t2hh))

tiempo_de_respuesta_HH(sample_conv_city_bell).iloc[0]

<class 'pandas.core.series.Series'>


Timedelta('0 days 00:03:25')

In [102]:
print (tiempo_de_respuesta_HH(sample_conv_city_bell))

# salud consultorios, conv id = 7 
print (tiempo_de_respuesta_HH(sample_conv_salud_c))


34   0 days 00:03:25
Name: SentDate, dtype: timedelta64[ns]
56   0 days 00:00:37
Name: SentDate, dtype: timedelta64[ns]


Ahora tenemos una muestra de como calcular el tiempo de respuesta para una conversación. En las siguientes secciones vamos a trabajar en filtrar los datos para considerar unicamente conversaciones con una ocurrencia de `Got_HH == 1`

### Boceto: Mensajes en la conversación

In [103]:
# TBD ¿Cómo podemos contar los mensajes en una conversación?

## Filtro de conversaciones

In [104]:
def with_human_handoff(df):
  """
  Filter conversations which where 'human handoffed'
  """
  c = df.groupby(by = ['conv_id', 'Fecha', 'Hospital'])[[ 'Got_HH']].sum()
  return c[c.Got_HH > 0]

In [105]:
hhs = with_human_handoff(df_handoffed)
len(hhs[hhs.Got_HH == 1]), len(hhs[hhs.Got_HH > 1])


(2472, 135)

In [106]:
# Nos quedamos únicamente con las conversaciones que tienen un único pasaje a humano
hhs = hhs[hhs.Got_HH == 1].reset_index().rename(columns = {"Got_HH": "Got_HH_sum"})
hhs

,conv_id,Fecha,Hospital,Got_HH_sum
0,7,2022-03-16,Salud Consultorios,1
1,10,2022-02-16,CityBell,1
2,11,2022-02-16,Salud Consultorios,1
3,13,2022-02-23,CityBell,1
4,17,2022-02-15,Salud Consultorios,1
...,...,...,...,...
2467,7410,2022-03-31,Salud Consultorios,1
2468,7422,2022-03-31,Salud Consultorios,1
2469,7424,2022-03-31,Salud Consultorios,1
2470,7438,2022-03-31,CityBell,1


### Seguimos mirando
A partir del índice del paso anterior podemos filtrar los grupos de interes en el dataframe general

In [107]:
x = df.merge(hhs, on = ['conv_id', 'Fecha', 'Hospital'])
x

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id,Got_HH_sum
0,CityBell,whatsapp:+5492215137770,Hola! Este es un mensaje automático. Me encuen...,received,2022-02-16 19:33:19+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
1,CityBell,whatsapp:+5492215137770,"Buenas tardes, por favor enviar los datos requ...",delivered,2022-02-16 19:33:17+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
2,CityBell,whatsapp:+5492215137770,Necesito hacerme un electrocardiograma,received,2022-02-16 19:22:00+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
3,CityBell,whatsapp:+5492215137770,"Fuiste transferido a un humano, por favor escr...",delivered,2022-02-16 19:21:47+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
4,CityBell,whatsapp:+5492215137770,Has seleccionado una práctica que requiere ate...,delivered,2022-02-16 19:21:47+00:00,2022-02-16,16,2,19,1,...,[],0,[],0,[],0,[],0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76403,CityBell,whatsapp:+5492215137770,"Necesito tu DNI, por favor.",read,2022-03-31 09:25:03+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76404,CityBell,whatsapp:+5492215137770,Sacar turno,received,2022-03-31 09:25:03+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76405,CityBell,whatsapp:+5492215137770,"Hola, soy el asistente virtual de Centro Medic...",read,2022-03-31 09:24:43+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1
76406,CityBell,whatsapp:+5492215137770,Tu sesión se ha reinciado por tiempo de inacti...,read,2022-03-31 09:24:43+00:00,2022-03-31,31,3,9,1,...,[],0,[],0,[],0,[],0,7235,1


In [108]:
x.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id',
       'Got_HH_sum'],
      dtype='object')

In [109]:
x[x.Got_HH == 1][['Got_HH','Got_HH_sum']]
x.sample(3)

,Hospital,Tel_hospital,Body,Status,SentDate,Fecha,Dia,Mes,Hora,Messages,...,alta_dni,not_DNI,issue_name,Issue_Name,ask_kunan,Ask_Kunan,many_fallbacks_goto_hh,Many_Fallbacks_Goto_HH,conv_id,Got_HH_sum
52116,CityBell,whatsapp:+5492215137770,"Necesito tu número de teléfono, por favor. Con...",read,2022-03-15 16:15:03+00:00,2022-03-15,15,3,16,1,...,[],0,[],0,[],0,[],0,3797,1
1867,Los Cedros,whatsapp:+5491120401203,Sacar turnos,received,2022-02-02 18:28:06+00:00,2022-02-02,2,2,18,1,...,[],0,[],0,[],0,[],0,387,1
12769,CityBell,whatsapp:+5492215137770,"Hola, soy el asistente virtual de Centro Medic...",delivered,2022-02-14 19:05:07+00:00,2022-02-14,14,2,19,1,...,[],0,[],0,[],0,[],0,2553,1


In [110]:
x.columns

Index(['Hospital', 'Tel_hospital', 'Body', 'Status', 'SentDate', 'Fecha',
       'Dia', 'Mes', 'Hora', 'Messages', 'Direction', 'Appointment_msp',
       'Appointment', 'Cancellation_msp', 'Cancelled', 'consult',
       'Consult_Appoint', 'fail_HH_sms', 'Fail_HH', 'achieve_HH', 'Got_HH',
       'Cupo', 'Full_turnos_obra_social', 'No_Relation', 'No_Correlation',
       'has_error', 'Error_Interno', 'has_error_501', 'Error_501',
       'falla_api_externo', 'Falla_Api_Externo', 'alta_dni', 'not_DNI',
       'issue_name', 'Issue_Name', 'ask_kunan', 'Ask_Kunan',
       'many_fallbacks_goto_hh', 'Many_Fallbacks_Goto_HH', 'conv_id',
       'Got_HH_sum'],
      dtype='object')

In [111]:
def momento(request_HH, posteriores):
    time_to_hh = SentDate_delta(request_HH, posteriores)
    if time_to_hh is None:
        return None
    
    
    time2HH = time_to_hh.iloc[0]

    hora = request_HH.Hora
    if 1 <= hora <= 5:
        return ('Madrugada', time2HH)
    elif 6 <= hora <= 12:
        return ('Mañana', time2HH)
    elif 13 <= hora <= 19:
        return ('Tarde', time2HH)
    elif (20 <= hora <= 23) or (hora == 0):
        return ('Noche', time2HH)
    
    print(f"no such time {hora}")
    return None
    
tiempo_de_respuesta_HH_con_momento = lambda data: hh_machinery(data, handler = momento)

def conv_digest(df):
    g = df.groupby(by = ['conv_id', 'Hospital', 'Fecha'])

    for each in list(g.groups)[:]:
        data = g.get_group(each).sort_values(by='SentDate')
        info = tiempo_de_respuesta_HH_con_momento(data)
        if info is None:
            continue
            
        momento, time2HH = info
        c = data.Messages.sum()
            
        yield each + (momento, time2HH, c)

    

In [112]:

col_names = ['conv_id', 'Hospital', 'Fecha', 'day_moment', 'Tiempo_de_respuesta_HH', 'Num_mensajes']

aggs = pd.DataFrame(conv_digest(x), columns = col_names)
aggs

,conv_id,Hospital,Fecha,day_moment,Tiempo_de_respuesta_HH,Num_mensajes
0,7,Salud Consultorios,2022-03-16,Tarde,0 days 00:00:37,24
1,10,CityBell,2022-02-16,Tarde,0 days 00:00:00,16
2,11,Salud Consultorios,2022-02-16,Mañana,0 days 09:07:25,31
3,13,CityBell,2022-02-23,Tarde,0 days 00:00:00,14
4,17,Salud Consultorios,2022-02-15,Mañana,0 days 00:20:09,53
...,...,...,...,...,...,...
2164,7403,CityBell,2022-03-31,Mañana,0 days 01:09:13,23
2165,7410,Salud Consultorios,2022-03-31,Mañana,0 days 00:56:37,22
2166,7422,Salud Consultorios,2022-03-31,Mañana,0 days 00:03:29,28
2167,7424,Salud Consultorios,2022-03-31,Mañana,0 days 00:05:42,26


In [113]:
aggs.describe()

,conv_id,Tiempo_de_respuesta_HH,Num_mensajes
count,2169.000000,2169,2169.000000
mean,3122.344398,0 days 00:27:37.650069156,31.394652
std,1998.569868,0 days 00:56:04.255917291,20.205519
min,7.000000,0 days 00:00:00,7.000000
25%,1373.000000,0 days 00:00:00,19.000000
50%,3055.000000,0 days 00:06:03,25.000000
75%,4596.000000,0 days 00:27:37,37.000000
max,7438.000000,0 days 09:37:49,224.000000


Vemos que existen algunos valores de tiempo de respuesta inmediatos (`0 segundos`). Analizamos un caso particular para entender que está pasando

In [114]:
aggs.sort_values(by='Tiempo_de_respuesta_HH')

,conv_id,Hospital,Fecha,day_moment,Tiempo_de_respuesta_HH,Num_mensajes
2168,7438,CityBell,2022-03-31,Mañana,0 days 00:00:00,13
1228,3467,CityBell,2022-03-14,Tarde,0 days 00:00:00,17
431,1088,Salud Consultorios,2022-02-11,Tarde,0 days 00:00:00,16
432,1089,CityBell,2022-02-04,Mañana,0 days 00:00:00,13
1227,3466,CityBell,2022-03-14,Tarde,0 days 00:00:00,16
...,...,...,...,...,...,...
2138,7295,CityBell,2022-03-31,Mañana,0 days 07:17:48,22
643,1699,CityBell,2022-03-22,Mañana,0 days 07:29:18,40
1200,3400,Salud Consultorios,2022-03-14,Mañana,0 days 08:38:48,39
2,11,Salud Consultorios,2022-02-16,Mañana,0 days 09:07:25,31


In [115]:
faulting = x[(x.conv_id == 3466) & (x.Hospital == 'CityBell') & (x.Fecha == '2022-03-14')]\
    .sort_values(by='SentDate')\
    .reset_index()
faulting
start = faulting[faulting.Got_HH == 1].index[0]


context = faulting[start-1:]

context[context.Direction == 'outbound-api'][['index', 'Got_HH', 'Body', 'SentDate']]


,index,Got_HH,Body,SentDate
13,49801,1,"Fuiste transferido a un humano, por favor escr...",2022-03-14 16:19:09+00:00
14,49802,0,Has seleccionado una práctica que requiere ate...,2022-03-14 16:19:09+00:00
15,49803,0,"Por la opción elegida, te transferiré con una ...",2022-03-14 16:19:09+00:00


Vamos a filtrar estos valores de nuestro conjunto de datos, aunque cumplen con el criterio provisto por la mentoria.

In [116]:
tiempo_de_respuesta_HH(faulting).iloc[0] == pd.Timedelta(value=0)

True

In [117]:

aggs = aggs[aggs.Tiempo_de_respuesta_HH != pd.Timedelta(value=0)].sort_values(by = 'Tiempo_de_respuesta_HH')
aggs.describe()

,conv_id,Tiempo_de_respuesta_HH,Num_mensajes
count,1520.000000,1520,1520.000000
mean,3087.291447,0 days 00:39:25.423026315,32.903947
std,1985.106197,0 days 01:03:25.088155911,19.996774
min,7.000000,0 days 00:00:01,14.000000
25%,1364.000000,0 days 00:05:04,21.000000
50%,2954.000000,0 days 00:15:47.500000,27.000000
75%,4504.000000,0 days 00:44:23.750000,37.000000
max,7424.000000,0 days 09:37:49,224.000000


### Dataset
Creamos el dataset para pasarle al regresor

In [118]:
join_columns = ['conv_id', 'Fecha', 'Hospital']

In [119]:
source = df.groupby(by=join_columns).sum().drop(labels = ['Dia', 'Mes', 'Hora'], axis = 1).reset_index()
source

,conv_id,Fecha,Hospital,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,No_Correlation,Error_Interno,Error_501,Falla_Api_Externo,not_DNI,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH
0,0,2022-02-16,Carra,75,1,0,0,0,0,4,0,0,0,0,0,0,0,0
1,0,2022-02-17,Carra,19,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,2022-02-19,Carra,25,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,0,2022-03-01,Carra,67,0,0,0,0,0,9,0,0,0,0,0,0,0,0
4,0,2022-03-25,Carra,22,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18659,7483,2022-03-31,Carra,32,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18660,7484,2022-03-31,Los Cedros,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18661,7485,2022-03-31,Los Cedros,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18662,7486,2022-03-30,Carra,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [120]:

dataset = source.merge(aggs, on = ['conv_id', 'Fecha', 'Hospital']).sort_values(by='conv_id')

In [121]:
dataset.columns

Index(['conv_id', 'Fecha', 'Hospital', 'Messages', 'Appointment', 'Cancelled',
       'Consult_Appoint', 'Fail_HH', 'Got_HH', 'Full_turnos_obra_social',
       'No_Correlation', 'Error_Interno', 'Error_501', 'Falla_Api_Externo',
       'not_DNI', 'Issue_Name', 'Ask_Kunan', 'Many_Fallbacks_Goto_HH',
       'day_moment', 'Tiempo_de_respuesta_HH', 'Num_mensajes'],
      dtype='object')

In [122]:
dataset.sample(10)

,conv_id,Fecha,Hospital,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,...,Error_Interno,Error_501,Falla_Api_Externo,not_DNI,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH,day_moment,Tiempo_de_respuesta_HH,Num_mensajes
1138,4492,2022-02-22,CityBell,17,0,0,0,0,1,0,...,0,0,0,1,0,0,0,Mañana,0 days 00:16:15,17
286,1060,2022-03-03,CityBell,41,0,0,0,1,1,0,...,0,0,0,2,0,0,0,Mañana,0 days 00:08:04,41
683,2635,2022-02-14,Salud Consultorios,44,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Tarde,0 days 00:25:35,44
147,491,2022-02-17,Salud Consultorios,25,0,0,0,0,1,0,...,0,0,0,1,0,0,0,Mañana,0 days 00:07:34,25
1215,4857,2022-03-21,CityBell,25,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Tarde,0 days 00:59:13,25
360,1298,2022-03-25,CityBell,48,0,0,0,0,1,0,...,0,0,0,0,0,0,1,Mañana,0 days 00:36:22,48
373,1348,2022-02-07,CityBell,21,0,0,0,0,1,0,...,0,0,0,1,0,0,0,Tarde,0 days 00:22:52,21
747,2886,2022-03-10,CityBell,29,0,0,0,1,1,0,...,0,0,0,2,0,0,0,Mañana,0 days 00:01:28,29
284,1045,2022-03-29,Salud Consultorios,40,1,0,0,0,1,0,...,0,0,0,0,0,0,0,Mañana,0 days 00:10:14,40
1226,4902,2022-02-24,Salud Consultorios,35,0,0,0,0,1,0,...,0,0,0,0,0,0,0,Tarde,0 days 00:01:35,35


In [123]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1520 entries, 0 to 1519
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   conv_id                  1520 non-null   int64          
 1   Fecha                    1520 non-null   object         
 2   Hospital                 1520 non-null   object         
 3   Messages                 1520 non-null   int64          
 4   Appointment              1520 non-null   int64          
 5   Cancelled                1520 non-null   int64          
 6   Consult_Appoint          1520 non-null   int64          
 7   Fail_HH                  1520 non-null   int64          
 8   Got_HH                   1520 non-null   int64          
 9   Full_turnos_obra_social  1520 non-null   int64          
 10  No_Correlation           1520 non-null   int64          
 11  Error_Interno            1520 non-null   int64          
 12  Error_501           

In [124]:
dataset.Fecha.sample(4)

1453    2022-03-30
484     2022-03-07
706     2022-03-10
1461    2022-03-31
Name: Fecha, dtype: object

----

### Regresion lineal

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error

Antes de entrenar el regresor vamos a codificar las varibles no numéricas
* Hospital
* Fecha
* Tiempo de respuesta

In [128]:
ordinal_enc = OrdinalEncoder()
encoded_hospital = ordinal_enc.fit_transform(dataset.Hospital.values.reshape(-1,1))
dataset['encoded_hospital'] = encoded_hospital
dataset

,conv_id,Fecha,Hospital,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,...,Error_501,Falla_Api_Externo,not_DNI,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH,day_moment,Tiempo_de_respuesta_HH,Num_mensajes,encoded_hospital
0,7,2022-03-16,Salud Consultorios,24,0,0,0,0,1,0,...,0,0,0,0,0,0,Tarde,0 days 00:00:37,24,2.0
1,11,2022-02-16,Salud Consultorios,31,0,0,0,0,1,0,...,0,0,2,0,0,0,Mañana,0 days 09:07:25,31,2.0
2,17,2022-02-15,Salud Consultorios,53,0,0,0,0,1,0,...,0,0,0,0,0,0,Mañana,0 days 00:20:09,53,2.0
3,21,2022-03-02,CityBell,62,1,0,0,0,1,0,...,0,0,1,0,0,0,Mañana,0 days 00:07:42,62,0.0
4,21,2022-03-07,CityBell,35,0,0,0,0,1,0,...,0,0,0,0,0,1,Tarde,0 days 00:07:26,35,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,7377,2022-03-31,Salud Consultorios,27,0,0,0,0,1,0,...,0,0,1,0,0,0,Tarde,0 days 00:10:31,27,2.0
1516,7403,2022-03-31,CityBell,23,0,0,0,0,1,0,...,0,0,1,0,0,1,Mañana,0 days 01:09:13,23,0.0
1517,7410,2022-03-31,Salud Consultorios,22,0,0,0,0,1,0,...,0,0,0,0,0,0,Mañana,0 days 00:56:37,22,2.0
1518,7422,2022-03-31,Salud Consultorios,28,0,0,0,0,1,0,...,0,0,0,0,0,0,Mañana,0 days 00:03:29,28,2.0


In [139]:
label_enc = LabelEncoder()
encoded_fecha = label_enc.fit_transform(dataset.Fecha.values.ravel())
encoded_fecha
dataset['encoded_fecha'] = encoded_fecha
dataset


,conv_id,Fecha,Hospital,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,...,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH,day_moment,Tiempo_de_respuesta_HH,Num_mensajes,encoded_hospital,encoded_fecha,seconds_to_HH,encoded_day_moment
0,7,2022-03-16,Salud Consultorios,24,0,0,0,0,1,0,...,0,0,0,Tarde,0 days 00:00:37,24,2.0,34,37.0,34
1,11,2022-02-16,Salud Consultorios,31,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 09:07:25,31,2.0,12,32845.0,12
2,17,2022-02-15,Salud Consultorios,53,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:20:09,53,2.0,11,1209.0,11
3,21,2022-03-02,CityBell,62,1,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:07:42,62,0.0,22,462.0,22
4,21,2022-03-07,CityBell,35,0,0,0,0,1,0,...,0,0,1,Tarde,0 days 00:07:26,35,0.0,26,446.0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,7377,2022-03-31,Salud Consultorios,27,0,0,0,0,1,0,...,0,0,0,Tarde,0 days 00:10:31,27,2.0,46,631.0,46
1516,7403,2022-03-31,CityBell,23,0,0,0,0,1,0,...,0,0,1,Mañana,0 days 01:09:13,23,0.0,46,4153.0,46
1517,7410,2022-03-31,Salud Consultorios,22,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:56:37,22,2.0,46,3397.0,46
1518,7422,2022-03-31,Salud Consultorios,28,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:03:29,28,2.0,46,209.0,46


In [141]:
label_enc = LabelEncoder()
encoded_day_moment = label_enc.fit_transform(dataset.day_moment.values.ravel())

dataset['encoded_day_moment'] = encoded_day_moment
dataset


,conv_id,Fecha,Hospital,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,...,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH,day_moment,Tiempo_de_respuesta_HH,Num_mensajes,encoded_hospital,encoded_fecha,seconds_to_HH,encoded_day_moment
0,7,2022-03-16,Salud Consultorios,24,0,0,0,0,1,0,...,0,0,0,Tarde,0 days 00:00:37,24,2.0,34,37.0,1
1,11,2022-02-16,Salud Consultorios,31,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 09:07:25,31,2.0,12,32845.0,0
2,17,2022-02-15,Salud Consultorios,53,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:20:09,53,2.0,11,1209.0,0
3,21,2022-03-02,CityBell,62,1,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:07:42,62,0.0,22,462.0,0
4,21,2022-03-07,CityBell,35,0,0,0,0,1,0,...,0,0,1,Tarde,0 days 00:07:26,35,0.0,26,446.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,7377,2022-03-31,Salud Consultorios,27,0,0,0,0,1,0,...,0,0,0,Tarde,0 days 00:10:31,27,2.0,46,631.0,1
1516,7403,2022-03-31,CityBell,23,0,0,0,0,1,0,...,0,0,1,Mañana,0 days 01:09:13,23,0.0,46,4153.0,0
1517,7410,2022-03-31,Salud Consultorios,22,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:56:37,22,2.0,46,3397.0,0
1518,7422,2022-03-31,Salud Consultorios,28,0,0,0,0,1,0,...,0,0,0,Mañana,0 days 00:03:29,28,2.0,46,209.0,0


In [131]:
seconds_to_HH = dataset.Tiempo_de_respuesta_HH.apply(lambda t2hh: t2hh.total_seconds())
dataset['seconds_to_HH'] = seconds_to_HH

In [132]:
numeric_dataset = dataset.drop(labels = ['day_moment', 'Hospital', 'Fecha', 'Tiempo_de_respuesta_HH'], axis = 1)
target = numeric_dataset.seconds_to_HH
features = numeric_dataset.drop(labels = 'seconds_to_HH', axis=1 )
features

,conv_id,Messages,Appointment,Cancelled,Consult_Appoint,Fail_HH,Got_HH,Full_turnos_obra_social,No_Correlation,Error_Interno,Error_501,Falla_Api_Externo,not_DNI,Issue_Name,Ask_Kunan,Many_Fallbacks_Goto_HH,day_moment,Num_mensajes,encoded_hospital,encoded_fecha
0,7,24,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Tarde,24,2.0,34
1,11,31,0,0,0,0,1,0,0,3,0,0,2,0,0,0,Mañana,31,2.0,12
2,17,53,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Mañana,53,2.0,11
3,21,62,1,0,0,0,1,0,0,0,0,0,1,0,0,0,Mañana,62,0.0,22
4,21,35,0,0,0,0,1,0,0,0,0,0,0,0,0,1,Tarde,35,0.0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1515,7377,27,0,0,0,0,1,0,0,0,0,0,1,0,0,0,Tarde,27,2.0,46
1516,7403,23,0,0,0,0,1,0,0,0,0,0,1,0,0,1,Mañana,23,0.0,46
1517,7410,22,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Mañana,22,2.0,46
1518,7422,28,0,0,0,0,1,0,0,0,0,0,0,0,0,0,Mañana,28,2.0,46


In [133]:
x_train, x_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state = 0)

regressor = LinearRegression(copy_X = True)
fitted_regressor = regressor.fit(x_train, y_train)

(fitted_regressor.coef_, fitted_regressor.intercept_)


ValueError: could not convert string to float: 'Mañana'

In [ ]:
predictions = fitted_regressor.predict(x_test)

mae = mean_absolute_error(predictions, y_test.values.reshape(-1,1))

rms = mean_squared_error(predictions, y_test.values.reshape(-1,1), squared=False)

list(map(lambda metric: pd.Timedelta(value=metric, unit = 'seconds'), [rms, mae] ))
    

In [ ]:
dataset.Tiempo_de_respuesta_HH.describe()